In [1]:
##############################################
# Task subdivision based on reachability     #
##############################################

# Packages
import numpy as np
import time
import rospy
import rospkg
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import HybridSegment
from corrective_shared_autonomy.TaskModels.DMPLWRhardcoded import DMPLWRhardcoded
from core_robotics.dataviz import threedplot, highlightedthreedplot
from core_robotics.PyBSpline import BSplineSurface
from scipy.spatial.transform import Rotation as ScipyR
from scipy.spatial.transform import Slerp

rospy.init_node('subdivisiontester', anonymous=True)

In [4]:
# TEST THE NEW ROUTINE
from corrective_shared_autonomy.TaskModels.FragmentedExecution import getFragmentedTraj, constructConnectedTraj, interpMultD


def plotReachable(trajectories,taskmask):
    x_good = []
    y_good = []
    z_good = []
    x_bad = []
    y_bad = []
    z_bad = []
    for ii in range(0,len(trajectories)):
        for jj in range(0,np.shape(trajectories[ii])[1]):
            if taskmask[ii][jj]==1:
                x_good.extend([trajectories[ii][0,jj]])
                y_good.extend([trajectories[ii][1,jj]])
                z_good.extend([trajectories[ii][2,jj]])
            else:
                x_bad.extend([trajectories[ii][0,jj]])
                y_bad.extend([trajectories[ii][1,jj]])
                z_bad.extend([trajectories[ii][2,jj]])

    x = np.array(x_good)
    y = np.array(y_good)
    z = np.array(z_good)
    xs = [x_bad]
    ys = [y_bad]
    zs = [z_bad]
    highlightedthreedplot(x, y, z, xs, ys, zs, marker_size = 5.0)

######### Set up data ###############
rospack = rospkg.RosPack()
uliconfigpath = rospack.get_path('uli_config')
config_dir = rospack.get_path('uli_config')+'/registration_models/'

# 0.43070987363200586
# 0.1
t_surf = np.array([0.1, -0.013269419259123479, 0.06385816493811966])
R_surf = np.array([0.008339958880240922, 0.004026424369577458, -0.029073916288389654, 0.9995343617826891])
surf_file = uliconfigpath + '/registration_models/IRC_piece1.csv'

ang_relative_to_surface = 90 # degrees
R_tool_surf = ScipyR.from_euler('zx',[ang_relative_to_surface,0],degrees=True)

surf = BSplineSurface()
surf.loadSurface(surf_file)


theta_q = R_tool_surf.as_quat()
state_names = ['u','v','f','theta_qx','theta_qy','theta_qz','theta_qw']
starting = [[0.1, 0.1], [0.3, 0.1], [0.5, 0.1], [0.7, 0.1], [0.9, 0.1]]
ending = [[0.1, 0.9], [0.3, 0.9], [0.5, 0.9], [0.7, 0.9], [0.9, 0.9]]
samps = [80,80,80,80,80]
samps_per_sec = 20

state_vals = []
for ii in range(0,len(starting)):
    start_temp = np.array([starting[ii][0],starting[ii][1],-10.0,theta_q[0],theta_q[1],theta_q[2],theta_q[3]])
    end_temp = np.array([ending[ii][0],ending[ii][1],-10.0,theta_q[0],theta_q[1],theta_q[2],theta_q[3]])
    state_vals.append(interpMultD(start_temp,end_temp,samps[ii]))
    
corrections = np.array([0, 0, 0, 0, 0, 0, 1])

######### Test behavior ###############
startt = time.time()
min_traj_length = 10
mask, trajs = getFragmentedTraj(surf,state_names,state_vals,R_surf,t_surf, min_traj_length)
print("TIME: ",time.time()-startt)

plotReachable(trajs,mask)

print(mask[2])
segments = constructConnectedTraj(surf,state_names,state_vals,mask,corrections,samps_per_sec)

# for ii in range(0,len(segments)):
#     print(segments[ii].state_names)
#     print(np.round(segments[ii].original_vals[0],2))

model = DMPLWRhardcoded(verbose=True, dt=1./samps_per_sec)
learnedSegments = model.learnModel(segments,config_dir+'subdivtest.pkl') # second argument is the outfile

TIME:  4.240057945251465
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1.]
-------------------
Model Learning: DMP + LWR Hardcoded
-------------------
DMP for segment  0  of  18
DMP for segment  1  of  18
DMP for segment  2  of  18
DMP for segment  3  of  18
DMP for segment  4  of  18
DMP for segment  5  of  18
DMP for segment  6  of  18
DMP for segment  7  of  18
DMP for segment  8  of  18
DMP for segment  9  of  18
DMP for segment  10  of  18
DMP for segment  11  of  18
DMP for segment  12  of  18
DMP for segment  13  of  18
DMP for segment  14  of  18
DMP for segment  15  of  18
DMP for segment  16  of  18
DMP for segment  17  of  18
DMP for segment  18  of  18
Time to learn model: 9.297756671905518 seconds
